## Monte Carlo Option Pricing Example Using QMCPy
This MATLAB script shows how to use the QMCPy to perform Monte Carlo option pricing.  The solution has a
more rigorous foundation than CLT confidence intervals. See *OptionPricingExample* for some of the background of this example for the
background of the problem.  See *OptionPricingMeanMC_CLT* for the solution using CLT confidence intervals.

In [1]:
import qmcpy as qp
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import time

/Users/yding/opt/anaconda3/envs/qmcpy/lib/python3.9/site-packages/scipy/__init__.py:155


In [2]:
S0 = 537.36 # initial stock price
timeFinal = 1/2  # half year to expiry
interest = 0.0050238 # interest
volatility = 0.19654 # volatility
SVal = lambda n: S0*np.exp((interest - volatility**2/2)*timeFinal + volatility*np.sqrt(timeFinal)*np.random.randn(n,1))
K = 600 # strike price
euroCallPayoff = lambda n: np.maximum(SVal(n) - K, 0) * np.exp(-interest * timeFinal) #discounted payoffs
trueEuroCallPrice = S0 * norm.cdf((np.log(S0/K) + (interest + volatility**2/2)*timeFinal)/(volatility * np.sqrt(timeFinal))) \
    - K * np.exp(-interest * timeFinal) * norm.cdf((np.log(S0/K)  + (interest - volatility**2/2)*timeFinal)/(volatility * np.sqrt(timeFinal)))


In [7]:
absTol = 0.05 # 5 cents absolute error
relTol = 0 # no relative error
alpha = 0.01 # Uncertainty 
start = time.time()
euroCallPrice,nSample = meanMC_CLT(euroCallPayoff,absTol,relTol,alpha)
end = time.time()
print("The option price = $",f"{ euroCallPrice:6.3f}"," +/- $", f"{ absTol:6.2f}", "with ", f"{int(nSample):,}","asset paths.")
print("The elpased time is {}".format(end-start))
print("The exact price is ", f"{trueEuroCallPrice:6.3f}")

The option price = $  9.869  +/- $   0.05 with  2,809,976 asset paths.
The elpased time is 0.12123394012451172
The exact price is   9.879


### How to use QMCPy to estimate the option price
There two methods we can call from QMCPy.
* qmcpy.CubMCCLT
* qmcpy.CubMCG

Before we use these two methods, we need to create an object of European Call option.

In [8]:
EuroCall = qp.EuropeanOption(qp.IIDStdUniform(dimension=1,seed=7), volatility= volatility,start_price= S0,
                       strike_price=K, interest_rate = interest,t_final=timeFinal,call_put='call')
print("The exact price of this European Call Option is ",f"{EuroCall.get_exact_value():.4f}")
sc = qp.CubMCCLT(EuroCall,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid asset paths, the estimation of the fair price is",f"{solution:.4f}")

The exact price of this European Call Option is  9.8791
To reach the absolute error tolerance =  0.05 , we need to generate  3,348,039 iid asset paths, the estimation of the fair price is 9.8695


In [9]:
sc = qp.CubMCG(EuroCall,absTol)
solution, data = sc.integrate()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid asset paths, the estimation of the fair price is",f"{solution:.4f}")

To reach the absolute error tolerance =  0.05 , we need to generate  3,839,027 iid asset paths, the estimation of the fair price is 9.8885


CubMCG is more conservative comparing to CubMCCLT.

#### meanMC_CLT function

In [3]:
def meanMC_CLT(inputRandomFuc,absTol,relTol,alpha): #Simple version of meanMC_CLT
    nsig = 1000 # initial size of the sample
    inflate = 1.2 # inflation rate
    YY = inputRandomFuc(nsig)
    sigma  = np.std(YY,ddof = 1)
    hum = np.mean(YY)
    sigmaUpBound = sigma * inflate #upper ound on the standard deviation
    nmu = max(1, np.power(np.ceil(norm.ppf(1-alpha/2)*sigmaUpBound/max(absTol,relTol)),2).astype(int) )  # number of samples needed for the error tolerance
    mu = np.mean(inputRandomFuc(nmu))
    nSample = nsig + nmu
    return mu, nSample